In [ ]:
import os
import pandas as pd

In [ ]:
# Ordnerpfad, wo sich die CSV-Dateien befinden
ordnerpfad = "C:/Users/anasl/Desktop/Daten 1/Daten/Monthly"

# Alle Dateinamen mit Endung ".csv" im Ordner sammeln
dateinamen = [datei for datei in os.listdir(ordnerpfad) if datei.endswith(".csv")]

In [4]:
# Leere Liste zum Speichern der eingelesenen DataFrames
alle_daten = []

for datei in dateinamen:
    pfad_zur_datei = os.path.join(ordnerpfad, datei)
    
    daten = pd.read_csv(
        pfad_zur_datei,
        sep=";",
        encoding="latin1"
    )
    
    alle_daten.append(daten)

In [5]:
# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenfügen
gesamt_daten = pd.concat(alle_daten, ignore_index=True)

In [6]:
gesamt_daten["Datum"] = pd.to_datetime(gesamt_daten["Datum"], errors="coerce")

In [7]:
# Mittelwert je Standort berechnen (für bestimmte Spalten)
spalten_mit_mittelwert = ["Boden", "MinTemp", "AvgTemp", "MaxTemp"]

# Fehlende Werte je Standort durch den jeweiligen Mittelwert ersetzen
for spalte in spalten_mit_mittelwert:
    gesamt_daten[spalte] = gesamt_daten.groupby("Standort")[spalte].transform(
        lambda gruppe: gruppe.fillna(gruppe.mean())
    )

# Fehlende Werte in „Sonne“ durch 0 ersetzen
gesamt_daten["Sonne"] = gesamt_daten["Sonne"].fillna(0)


In [8]:
# Die Spalten, die am Ende enthalten sein müssen
ziel_spalten = [
    "Standort", "Datum", "MinTemp", "AvgTemp", "MaxTemp",
    "Regen", "Sonne", "Mittel_Windstaerke", "Max_Windgeschwindigkeit", "Boden"
]

# Fehlende Spalten hinzufügen, falls sie nicht existieren
for spalte in ziel_spalten:
    if spalte not in gesamt_daten.columns:
        gesamt_daten[spalte] = pd.NA

# Spaltenreihenfolge korrekt anordnen
gesamt_daten = gesamt_daten[ziel_spalten]


In [ ]:
# Exportieren als CSV mit Komma als Trennzeichen und ohne Index
export_pfad = os.path.join(ordnerpfad, "Monthly_combined.csv")

gesamt_daten.to_csv(
    export_pfad,
    sep=",",
    index=False,
    encoding="latin1"
)